In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Function

In [2]:
!pip install imbalanced-learn
!pip install ipywidgets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.8 MB/s eta 0:00:00


In [3]:
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [4]:
def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)

def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)

def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  
  return dic


def models_check_box(models):
  from IPython.display import display
  new_keys=[]
  for i in models:
    i=widgets.Checkbox(
      value=False,
      description=str(i),
      disabled=False,
      indent=False
      )
    display(i)
    new_keys.append(i)
  return new_keys


def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)


def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)

#Starting

In [5]:
dataset = pd.read_csv("/content/drive/MyDrive/Sleep Stage 5 class/EEG_HMC_FeatureExtraction_2023.01.19.csv")
print(dataset.shape)
dataset.dropna(axis = 0,how='any', inplace = True)
dataset = dataset.reset_index(drop = True)
"""df.to_csv("New_EEG_NullValueRemoved_HMC.csv",index = False)
dataset = pd.read_csv("New_EEG_NullValueRemoved_HMC.csv")"""
target = "Sleep Stage"
result = {}
print(dataset.shape)

#Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

(108920, 52)
(108450, 52)


###Spliting into X and y

In [6]:
X =  dataset.loc[:,dataset.columns != target]  # removing Sleep Stage
X =  X.loc[:,X.columns != "Subject"]            # removing Status
X =  X.loc[:,X.columns != "Epoch"]             # removing Epoch
y = dataset[target]

#Tunning for Scalling and Data Balancing

##Smote : NO
##Scaling : NO

In [8]:
def tune_1():
  X_new = X
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (Standard)

In [9]:
def tune_2():
  X_new = StandardScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (MinMax)

In [10]:
def tune_3():
  X_new = MinMaxScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : Train (Stadard)

In [11]:
def tune_4():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : NO
##Scaling : Train (MinMax)

In [12]:

def tune_5():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : ALL
##Scaling : NO

In [13]:

def tune_6():
  X_new,y_new = Xy_balance(X,y)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)


##Smote : Train
##Scaling : NO

In [14]:

def tune_7():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  return X_train, X_test, y_train, y_test


##Smote : Train
##Scaling : Train (Standard)

In [15]:
def tune_8():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : Train
##Scaling : Train (MixMax)

In [16]:
def tune_9():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : Train (Standard)

In [17]:
def tune_10():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : Train (MixMax)

In [18]:
def tune_11():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : All (Standard)

In [19]:
def tune_12():
  X_new,y_new = Xy_balance(X,y)
  X_new = StandardScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : All
##Scaling : ALL (MinMax)

In [20]:
def tune_13():
  X_new,y_new = Xy_balance(X,y)
  X_new = MinMaxScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Tunn Dictionary

In [21]:
tune_dic={
  "Smote : NO    ; Scaling : NO"                  : tune_1(),
  "Smote : NO    ; Scaling : All Data(Standard)"  : tune_2(),
  "Smote : NO    ; Scaling : All Data(MinMax)"    : tune_3(),
  "Smote : NO    ; Scaling : Train(Standard)"     : tune_4(),
  "Smote : NO    ; Scaling : Train(MinMax)"       : tune_5(),
  "Smote : ALL   ; Scaling : NO"                  : tune_6(),
  "Smote : Train ; Scaling : NO"                  : tune_7(),
  "Smote : Train ; Scaling : Train(Standard)"     : tune_8(),
  "Smote : Train ; Scaling : Train(MinMax)"       : tune_9(),
  "Smote : All   ; Scaling : Train(Standard)"     : tune_10(),
  "Smote : All   ; Scaling : Train(MinMax)"       : tune_11(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_12(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_13()
}



---



#XGB

In [31]:
%%time
import xgboost as xgb
li = ["Smote : NO    ; Scaling : Train(Standard)",
      "Smote : NO    ; Scaling : Train(MinMax)",
      "Smote : Train ; Scaling : Train(Standard)",
      "Smote : Train ; Scaling : Train(MinMax)",
      "Smote : All   ; Scaling : Train(Standard)",
      "Smote : All   ; Scaling : Train(MinMax)"]
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)

xgb={}

for i in list(tune_dic.keys()):

  X_train, X_test, y_train, y_test = tune_dic[i]

  if i in li:
    xgb_deafult.fit(X_train,y_train.values)
    y_pred = xgb_deafult.predict(X_test.values)
  else:
    xgb_deafult.fit(X_train,y_train)
    y_pred = xgb_deafult.predict(X_test)


  from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
  accu = accuracy_score(y_test, y_pred)
  print(i)
  print("Accurecy: ",accu)
  xgb[i] = accu
  print("------------------------------------------------------------------------------")
  print("------------------------------------------------------------------------------")

df = pd.DataFrame.from_dict(xgb, orient="index")
df.to_csv("xgb.csv")

Smote : NO    ; Scaling : NO
Accurecy:  0.6432918395573998
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.6448132780082988
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.6432918395573998
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : Train(Standard)
Accurecy:  0.17671738128169664
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : Train(MinMax)
Accurecy:  0.4284001844167819
------------------

In [24]:
li = list(tune_dic.keys())

In [28]:
%%time

X_new = X
y_new = y
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
X_train = StandardScaleData(X_train)

CPU times: user 96.8 ms, sys: 970 µs, total: 97.7 ms
Wall time: 108 ms


In [30]:
%%time

xgb_deafult.fit(X_train,y_train.values)
y_pred = xgb_deafult.predict(X_test.values)

CPU times: user 1min 43s, sys: 116 ms, total: 1min 44s
Wall time: 55 s
